In [13]:
import numpy as np
import pandas as pd

In [14]:
names =['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

df = pd.read_csv('train.csv',header=None,sep=",",names=names, encoding = "ISO-8859-1")

In [15]:
print(df)

                      id                                       comment_text  \
0                     id                                       comment_text   
1       0000997932d777bf  Explanation\nWhy the edits made under my usern...   
2       000103f0d9cfb60f  D'aww! He matches this background colour I'm s...   
3       000113f07ec002fd  Hey man, I'm really not trying to edit war. It...   
4       0001b41b1c6bb37e  "\nMore\nI can't make any real suggestions on ...   
5       0001d958c54c6e35  You, sir, are my hero. Any chance you remember...   
6       00025465d4725e87  "\n\nCongratulations from me as well, use the ...   
7       0002bcb3da6cb337       COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK   
8       00031b1e95af7921  Your vandalism to the Matt Shirvington article...   
9       00037261f536c51d  Sorry if the word 'nonsense' was offensive to ...   
10      00040093b2687caa  alignment on this subject and which are contra...   
11      0005300084f90edc  "\nFair use rationale for 

In [16]:
df.head(5)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
1,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
2,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
3,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
4,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0


In [17]:
df['toxic_score'] = df[['toxic','severe_toxic','obscene', 'threat', 'insult', 'identity_hate']].max(axis=1)
df = df.drop(['id', 'toxic','severe_toxic','obscene', 'threat', 'insult', 'identity_hate'], axis=1)
print(df)

                                             comment_text toxic_score
0                                            comment_text       toxic
1       Explanation\nWhy the edits made under my usern...           0
2       D'aww! He matches this background colour I'm s...           0
3       Hey man, I'm really not trying to edit war. It...           0
4       "\nMore\nI can't make any real suggestions on ...           0
5       You, sir, are my hero. Any chance you remember...           0
6       "\n\nCongratulations from me as well, use the ...           0
7            COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK           1
8       Your vandalism to the Matt Shirvington article...           0
9       Sorry if the word 'nonsense' was offensive to ...           0
10      alignment on this subject and which are contra...           0
11      "\nFair use rationale for Image:Wonju.jpg\n\nT...           0
12      bbq \n\nbe a man and lets discuss it-maybe ove...           0
13      Hey... what 